**1. Analisando e Prevendo Doação de Sangue**

O objetivo deste projeto é analisar e criar um modelo de aprendizado de máquina em relação a uma base de dados de doares de sangue em uma unidade móvel de coleta em Taiwan. 

O Centro de Serviço de Tranfusão de Sangue (em Inglês, *Blood Transfusion Service Cente*), a qual esta base de dados foi obtida, basicamenmte dirige-se a diversas universidades e realiza a coleta de alunos, professores e demais membros da universidade. Assim, o objetivo deste modelo é tentar prever se um determinado doador doará sangue novamente quando a unidade móvel visitar a mesma universidade, ou seja, é um problema de classificação do tipo binário (apenas duas possibilidades), em que 1 siginifica que ele doará e 0 que não.

A base de dados consiste em um arquivo estruturado no formato CSV, o que facilita sua análise e manipulação utilizando a biblioteca Pandas. Então, a primeira tarefa é analisar a formatação do arquivo, bem como a organização das colunas.







In [104]:
# Verificando a estrutura inicial do arquivo.
!head -n 5 doacoes.csv

Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),"whether he/she donated blood in March 2007"
2 ,50,12500,98 ,1
0 ,13,3250,28 ,1
1 ,16,4000,35 ,1
2 ,20,5000,45 ,1


Como se verifica acima, é um arquivo CSV bem estruturado e separado por virgulas. O proximo passo é identificar a quantidade de entradas (linhas) nele:

In [105]:
# Verificando o número de linhas do arquivo.
!wc -l doacoes.csv

748 doacoes.csv


Desconsiderando a primeira linha, que contém o nome das colunas, esse banco de dados possui 747 entradas.

**2. Importando e Organizando o Banco de Dados**

Como se viu anteriormente, o arquivo está bem estruturado e pode ser importando utilizando Pandas.

In [106]:
import pandas as pd

# Importa o arquivo CSV como um DataFrame.
dados = pd.read_csv('doacoes.csv',  sep=',')

# Exibe informações sobre o banco de dados.
dados.info()

# Exibe as primeiras 5 linhas.
dados.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 5 columns):
Recency (months)                              748 non-null int64
Frequency (times)                             748 non-null int64
Monetary (c.c. blood)                         748 non-null int64
Time (months)                                 748 non-null int64
whether he/she donated blood in March 2007    748 non-null int64
dtypes: int64(5)
memory usage: 29.3 KB


,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


Como se vê acima, os dados foram importados corretamente e todas as colunas já são valores numéricos, o que é um requisito para os algoritmos de aprendizagem de máquina. Além disso, percebe que as quatro primeiras colunas são as *features* e a última é a *label*, isto é, o valor binário ou booleano que representa o fato do indivíduo doar novamente ou não. 

Então, para melhorar a apresentação dos dados, as colunas serão renomeadas a seguir.



In [107]:
# Renomeia todas as colunas.
dados.rename(columns = {
      'Recency (months)':'num_meses_ultima_doacao',
      'Frequency (times)':'num_vezes',
      'Monetary (c.c. blood)': 'valor_recebido',
      'Time (months)': 'meses',
      'whether he/she donated blood in March 2007': 'doara_novamente',
    },
    inplace = True
)

# Exibe a norma estrutura de nomes das colunas.
dados.head(5)

,num_meses_ultima_doacao,num_vezes,valor_recebido,meses,doara_novamente
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


**3. Analisando os Dados em Detalhes**

O passo seguinte é verificar como é a distribuição da *label* de interesse, como forma de decidir como o processamento dos dados será feito.

In [108]:
# Exibie a distribuição dos dados em relação a coluna alvo.
dados.doara_novamente.value_counts(normalize=True)

0    0.762032
1    0.237968
Name: doara_novamente, dtype: float64

Como se vê, a quantidade de não reincidência de doação é muito maior, logo durante a divisão dos conjuntos treinamento e teste isto deve ser levado em consideração.

O passo seguinte é verificar a variância da base de dados.

In [109]:
# Exibe a variância.

dados.var().round(3)

num_meses_ultima_doacao         65.535
num_vezes                       34.098
valor_recebido             2131094.230
meses                          594.224
doara_novamente                  0.182
dtype: float64

Como se vê, a ordem de grandeza da *feature* valor_recebido é muito maior que as demais. Logo, é interessante realizar a normalização desta coluna.

In [110]:
import numpy as np

# Cria uma nova coluna normalizando os valores recebidos.
dados['valor_recebido_norm'] = np.log(dados.valor_recebido)

# Remove a coluna não normalizada.
dados.drop(columns=('valor_recebido'), inplace=True)

# Calcula a variância novamente.
dados.var().round(3)

num_meses_ultima_doacao     65.535
num_vezes                   34.098
meses                      594.224
doara_novamente              0.182
valor_recebido_norm          0.836
dtype: float64

**4. Preparando os Conjuntos de Treinamento e de Teste**

O primeiro passo durante a preparação dos conjuntos de treinamento e este é verificar quais *features* possuem maior relevância. Esse processo é feito a seguir:

Com os resultados acima, é possível criar os conjuntos de treinamento e teste.

In [111]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# Cria o X e o y.
X = dados.drop(labels="doara_novamente", axis=1)
y = dados.doara_novamente

test = SelectKBest(score_func=chi2, k='all')
test.fit(X, y)

# Exibe o resultado.
test.scores_

array([403.34516772, 220.77712112,  16.64519234,   4.56146679])

Como se vê, os dois primeiros *features* possuem muito mais relevância no conjunto de dados (ordem de grandeza muito maior que os dois últimos). Logo, os dois últimos podem ser descartados neste ponto.

In [0]:
X.drop(columns=['valor_recebido_norm', 'meses'], axis=1, inplace=True)

In [113]:
from sklearn.model_selection import train_test_split

# Cria os conjuntos de treinamento e teste.
X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                    test_size=0.20,
                                                    random_state=42,
                                                    shuffle=True,
                                                    stratify=dados.doara_novamente)

# Exibe a estrutura basica do X de treinamento
X_train.head(5)



,num_meses_ultima_doacao,num_vezes
529,2,6
271,16,7
455,21,1
175,11,10
309,16,3


**5. Selecionando o Modelo de Aprendizagem de Máquina**

Para se escolher o modelo de aprendizagem de máquina, serão avaliados os seguintes modelos:
- Árvore de Decisão
- KNN
- Regressão Logística

Todos os modelos acima passão por um processo de otimização dos hiper-parâmetros com o intuito de se obter o melhor modelo ao final baseando-se 

In [114]:
# Modelos a serem testados e escolhidos
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import accuracy_score

# Definindo os espaços de busca para os hiper parâmetros.
dt_param_grid = {
  'max_depth': np.arange(1, 10, 1),
  'criterion': ('gini', 'entropy'),
}

knn_param_grid = {
    'n_neighbors': np.arange(1, 21, 2),
    'weights': ('uniform', 'distance')
}

lr_param_grid = {
    'penalty': ('l1', 'l2'),
    'C': np.logspace(0, 4, 10),
    'solver': ('liblinear',),
}

# Define a validação cruzada de tamanho 5.
kf = KFold(5, shuffle=True, random_state=1)

# Resultados para a árvore de decisão.
dt_grid = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=kf, scoring='accuracy')
dt_best_model = dt_grid.fit(X_train, y_train)
dt_preds = dt_best_model.predict(X_test)

print("[DecisionTreeClassifier] Melhor score: %f com os parâmetros %s," % (dt_best_model.best_score_, dt_best_model.best_params_))
print("[DecisionTreeClassifier] Score do conjunto treinamento: ", accuracy_score(y_test, dt_preds))

# Resultados para o KNN.

knn_grid = GridSearchCV(KNeighborsClassifier(), knn_param_grid, cv=kf, scoring='accuracy')
knn_best_model = knn_grid.fit(X_train, y_train)
knn_preds = knn_best_model.predict(X_test)

print("[KNeighborsClassifier] Melhor score: %f com os parâmetros %s," % (knn_best_model.best_score_, knn_best_model.best_params_))
print("[KNeighborsClassifier] Score do conjunto treinamento: ", accuracy_score(y_test, knn_preds))

# Resultados para Regressão Logística.
lr_grid = GridSearchCV(LogisticRegression(), lr_param_grid, cv=kf, scoring='accuracy')
lr_best_model = lr_grid.fit(X_train, y_train)
lr_preds = lr_best_model.predict(X_test)

print("[LogisticRegression] Melhor score: %f com os parâmetros %s," % (lr_best_model.best_score_, lr_best_model.best_params_))
print("[LogisticRegression] Score do conjunto treinamento: ", accuracy_score(y_test, lr_preds))

[DecisionTreeClassifier] Melhor score: 0.767559 com os parâmetros {'criterion': 'entropy', 'max_depth': 3},
[DecisionTreeClassifier] Score do conjunto treinamento:  0.7733333333333333
[KNeighborsClassifier] Melhor score: 0.765886 com os parâmetros {'n_neighbors': 17, 'weights': 'distance'},
[KNeighborsClassifier] Score do conjunto treinamento:  0.78
[LogisticRegression] Melhor score: 0.765886 com os parâmetros {'C': 1.0, 'penalty': 'l1', 'solver': 'liblinear'},
[LogisticRegression] Score do conjunto treinamento:  0.7733333333333333


**6. Conclusão**

Com base nos resultados acima, nota-se que todos os modelos apresentaram resultados similares, com acurácia em torno de 77% para todos os casos. Porém, o modelo recomendado neste caso é o de Regressão Logística por possuir menor custo computacional.

In [115]:
from sklearn.metrics import confusion_matrix

final_model = LogisticRegression(C=1.0, penalty='l1', solver='liblinear')
final_model.fit(X, y)
preds = final_model.predict(X_test)

confusion_matrix(y_test, preds)

array([[113,   1],
       [ 33,   3]])

O resultado final mostra que nesse conjunto de dados houve apenas 1 resultado falso positivo (um usuário doaria mas não doou) e 33 como falso negativo (um usuário que não doaria mas doou sangue). Contudo, a grande maioria dos indivíduos foram classificados corretamente (113 esperados para doarem doaram e 3 esperados para não doarem não doaram de fato).

Portanto, o modelo acima mostra-se razoavelmente viável na predição de doadores para a unidade móvel acima e pode ser utilizado para prever a quantidade de material a ser levado para o dia da coleta, bem como estimar o número de funcionários necessários por exemplo.